In [ ]:
%load_ext autoreload

In [ ]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [ ]:
import logging
drake_logger = logging.getLogger("drake")
# drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
locs += np.array([0,0,-0.70])
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    print(q)
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


In [ ]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

In [ ]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat1.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

In [ ]:
visualizer.meshcat2.Delete("/prm")

In [ ]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle = partial(plot_prm, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = visualizer.col_func_handle_rational)


num_points = 20 #1000

PRM = prm.PRM( 
            limits_t,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

PRM_bad = prm.PRM( 
            limits_t,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle_bad,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

# PRM.add_start_end(start, target)
if num_points < 100:
    PRM.plot()
    PRM_bad.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
# path, sp_length = PRM.find_shortest_path()

# mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
# mat.wireframeLinewidth = 2.0
# num_waypoints = len(path)
# for idx in range(num_waypoints-1):
#     vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
# traj= utils.PWLinTraj(path, 5.0)

## Set up the sliders so we can move the plant around manually

In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

In [ ]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


In [ ]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, np.zeros(2),
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

In [ ]:
def make_prm_line_pairs(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    s0 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    s1 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0[i] = PRM.nodes[idx0]
        s1[i] = PRM.nodes[idx1]
    return s0, s1
s0_mat_safe, s1_mat_safe = make_prm_line_pairs(PRM)
s0_mat_unsafe, s1_mat_unsafe = make_prm_line_pairs(PRM_bad)
print(s0_mat_safe.shape)
print(s0_mat_unsafe.shape)

In [ ]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe

In [ ]:
def make_certifier_and_certify(use_affine = True):
    t0 = time.time()
    plane_type = rational_forward_kinematics.SeparatingPlaneOrder.kAffine if use_affine \
                    else rational_forward_kinematics.SeparatingPlaneOrder.kConstant
    line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                    plane_type, 
                                    np.zeros(2),
                                    set(), VerificationOption()
                                   )
    t1 = time.time()
    print(f"Time to construct line certifier = {t1-t0}s")
    
    t0 = time.time()
    safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_safe, s1_mat_safe)
    t1 = time.time()
    print(f"Certification of safe PRM in {t1-t0}s")
    num_safe1 = count_num_safe(safe_points)
    print(f"Certified {num_safe1}/{len(safe_points)} as safe")
    print(f"Failed to certify {len(safe_points)-num_safe1}/{len(safe_points)} safe points as safe")
    
    t0 = time.time()
    unsafe_points, unsafe_planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_unsafe, s1_mat_unsafe)
    t1 = time.time()
    print(f"Certification of unsafe PRM in {t1-t0}s")
    num_safe2 = count_num_safe(safe_points)
    print(f"Incorrectly certified {num_safe2}/{len(safe_points)} as safe")
    print(f"Classified {len(safe_points)-num_safe2}/{len(safe_points)} unsafe points as unsafe")
    return (safe_points, planes, num_safe1), (unsafe_points, unsafe_planes, num_safe2)

In [ ]:
ret_safe_affine, ret_unsafe_affine = make_certifier_and_certify(use_affine = True)